#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [1]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2017-01-30 18:14:27,497] Making new env: FrozenLake-v0


In [2]:
# display the game state
env.render()

SFFF
FHFH
FFFH
HFFG



### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print("initial observation code:",env.reset())
print('printing observation:')
env.render()
print("observations:",env.observation_space, 'n=',env.observation_space.n)
print("actions:",env.action_space, 'n=',env.action_space.n)

initial observation code: 0
printing observation:
SFFF
FHFH
FFFH
HFFG

observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [4]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:",new_obs)
print("reward:", reward)
print("is game over?:",is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 4
reward: 0.0
is game over?: False
printing new state:
SFFF
FHFH
FFFH
HFFG
  (Right)


In [5]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [6]:
env.reset()
env.step(action_to_i['down'])
env.step(action_to_i['down'])
env.step(action_to_i['right'])
env.step(action_to_i['right'])
env.step(action_to_i['down'])
env.step(action_to_i['right'])
env.render()

SFFF
FHFH
FFFH
HFFG
  (Right)


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [7]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0, n_actions, n_states)

In [8]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies,return_counts=True)[-1] /10**4. /n_states
print ("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas,[1./n_actions]*n_actions,atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print ("Seems fine!")

Action frequencies over 10^4 samples: [ 0.25014375  0.25130625  0.2495375   0.2490125 ]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [9]:
def sample_reward(env,policy,t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    """
    s = env.reset()
    total_reward = 0
    
    for i in range(t_max):
        s, reward, is_done, _ = env.step(policy[s])
        total_reward += reward
        if is_done:
            return total_reward
    return total_reward

In [10]:
print ("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int,float) for r in rewards]), 'sample_reward must return a single number'
assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake (if solving taxi, delete this line)'
print ("Looks good!")

generating 10^3 sessions...
Looks good!


In [11]:
def evaluate(policy,n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for i in range(n_times)]
    return np.mean(rewards)
        

In [12]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way"""
    lake = "SFFFFHFHFFFHHFFG"
    assert env.spec.id == "FrozenLake-v0","this function only works with frozenlake 4x4"
    
    # where to move from each tile
    arrows = ['<v>^'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow,tile in zip(arrows,lake)]
    
    for i in range(0,16,4):
        print (' '.join(signs[i:i+4]))

print ("random policy:")
print_policy(get_random_policy())

random policy:
> < < >
v H > H
> ^ > H
H > < G


### Random search

In [16]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(1000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print ("New best score:",score)
        print ("Best policy:")
        print_policy(best_policy)


  1%|          | 6/1000 [00:00<00:44, 22.47it/s]

New best score: 0.0
Best policy:
v v > ^
< H ^ H
> ^ v H
H < < G
New best score: 0.01
Best policy:
> ^ v >
^ H v H
v > < H
H < v G



  1%|▏         | 14/1000 [00:00<00:37, 26.41it/s]

New best score: 0.04
Best policy:
^ ^ > >
v H v H
< ^ < H
H v ^ G



  2%|▎         | 25/1000 [00:00<00:27, 34.93it/s]

...


  6%|▋         | 63/1000 [00:01<00:28, 32.64it/s]

New best score: 0.37
Best policy:
< ^ < v
< H < H
^ v < H
H ^ v G



 89%|████████▉ | 889/1000 [00:29<00:05, 20.46it/s]

New best score: 0.75
Best policy:
< > ^ >
< H > H
^ v < H
H > v G



100%|██████████| 1000/1000 [00:32<00:00, 30.82it/s]

# Part II Genetic algorithm

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [179]:
def crossover(policy1,policy2,p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    policy = []
    for i in range(n_states):
        if np.random.rand() >= p:
            policy.append(policy1[i])
        else:
            policy.append(policy2[i])
    return policy

In [180]:
def mutation(policy,p=0.1):
    """
    for each state, with probability p replace action with random action
    """
    return crossover(policy,get_random_policy(),p=p)
    

In [181]:
np.random.seed(1234)
policies = [crossover(get_random_policy(),get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'
print ("Seems fine!")

Seems fine!


In [182]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [183]:
print ("initializing...")
pool = [get_random_policy() for i in range(pool_size)] # <spawn a list of pool_size random policies>
pool_scores = [evaluate(policy) for policy in pool] # <evaluate every policy in the pool, return list of scores>


initializing...


In [184]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float,int, np.float64) for score in pool_scores])


In [185]:
#main loop
for epoch in range(n_epochs):
    print ("Epoch %s:"%epoch, flush=True)
    
    
    crossovered = [crossover(pool[np.random.randint(0, len(pool))],
                             pool[np.random.randint(0, len(pool))]) for i in range(n_crossovers)] 
        # <crossover random guys from pool, n_crossovers total>
    mutated = [mutation(pool[np.random.randint(0, len(pool))]) for i in range(n_mutations)]
        # <add several new policies at random, n_mutations total>
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool = pool + crossovered + mutated # <add up old population with crossovers/mutations>
    pool_scores = [evaluate(policy) for policy in pool] # <evaluate all policies again>
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:",pool_scores[-1], flush=True)
    print_policy(pool[-1])

Epoch 0:
best score: 0.17
^ ^ v ^
^ H < H
^ > v H
H > > G
Epoch 1:
best score: 0.21
^ ^ v ^
^ H < H
^ > v H
H > > G
Epoch 2:
best score: 0.2
^ ^ < v
> H < H
v ^ > H
H ^ v G
Epoch 3:
best score: 0.26
^ ^ v ^
^ H < H
^ > < H
H > > G
Epoch 4:
best score: 0.25
^ ^ v ^
^ H < H
^ > < H
H > > G
Epoch 5:
best score: 0.44
v ^ > ^
< H > H
^ v < H
H > > G
Epoch 6:
best score: 0.45
> ^ < ^
< H < H
^ v < H
H > ^ G
Epoch 7:
best score: 0.6
> ^ ^ ^
< H > H
^ v < H
H > v G
Epoch 8:
best score: 0.68
< > > ^
< H > H
^ v < H
H > v G
Epoch 9:
best score: 0.75
< > > ^
< H > H
^ v < H
H > v G
Epoch 10:
best score: 0.77
< > > ^
< H > H
^ v < H
H > v G
Epoch 11:
best score: 0.77
< ^ ^ ^
< H > H
^ v < H
H > v G
Epoch 12:
best score: 0.78
< ^ ^ ^
< H < H
^ v < H
H > v G
Epoch 13:
best score: 0.84
< > > ^
< H > H
^ v < H
H > v G
Epoch 14:
best score: 0.81
< ^ < ^
< H < H
^ v < H
H > v G
Epoch 15:
best score: 0.85
< > > ^
< H > H
^ v < H
H > v G
Epoch 16:
best score: 0.83
< ^ < >
< H > H
^ v < H
H > v G
Epoch 17:

KeyboardInterrupt: 

## moar

The parameters of the genetic algorithm aren't optimal, try to find something better. (size, crossovers and mutations)

Try alternative crossover and mutation strategies
* prioritize crossover for higher-scorers?
* try to select a more diverse pool, not just best scorers?
* Just tune the f*cking probabilities.

See which combination works best!

# Part III

The frozenlake problem above is just too simple: you can beat it even with a random policy search. Go solve something more complicated.

* __FrozenLake8x8-v0__ - frozenlake big brother (achieve score >0.7)
* __Taxi-v1__ - essentially a maze where you get score for moving passengers to their destinations. (achieve score >5)


### Some tips:
* Random policy search is worth trying as a sanity check, but in general you should expect the genetic algorithm (or anything you devised in it's place) to fare much better that random.
* While _it's okay to adapt the tabs above to your chosen env_, make sure you didn't hard-code any constants there (e.g. 16 states or 4 actions).
* `print_policy` function was built for the frozenlake-v0 env so it will break on any other env. You could simply ignore it OR write your own visualizer for bonus points.
* in function `sample_reward`, __make sure t_max steps is enough to solve the environment__ even if agent is sometimes acting suboptimally. To estimate that, run several sessions without time limit and measure their length.

### Bonus I:
* Gym envs have a condition for "beating the game". E.g. here's the conditions for [Taxi-v1](https://gym.openai.com/envs/Taxi-v1). 
 * If you managed to do that, it's worth uploading your first solution to gym. See `gym.upload(...)` docs. Allbeit it isn't a strong AI (or is it?), uploading your algorithm would be a good start. (and a +point!)

### Bonus II:
* There are environments with continuous state spaces. In fact, most real world environments have this property. While we will dive into methods designed for that later, right now you already can solve them through binarization.
 * Gym has a basic infinite-state-space env called [CartPole](https://gym.openai.com/envs/CartPole-v0) - please start from this one. Solving something more challenging is great, but make sure your algorithm beats cartpole first. Also kudos for submitting.
 * Main idea: if you have something infinite and you want something discrete, you split it into bins. Like what histogram does.
 * Good choice of discretes is critical!
 * If the dimensionality is too high, you can try to reduce it (PCA/autoencoders)



In [18]:
env = gym.make("Taxi-v2")

#start new game
env.reset();

[2017-01-30 18:17:33,296] Making new env: Taxi-v2


In [19]:
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [20]:
print("initial observation code:",env.reset())
print('printing observation:')
env.render()
print("observations:",env.observation_space, 'n=',env.observation_space.n)
print("actions:",env.action_space, 'n=',env.action_space.n)

initial observation code: 491
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

observations: Discrete(500) n= 500
actions: Discrete(6) n= 6


In [21]:
n_states = env.observation_space.n
n_actions = env.action_space.n

In [24]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(1000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print ("New best score:",score)
        # print ("Best policy:")
        # print_policy(best_policy)


  0%|          | 2/1000 [00:00<03:59,  4.18it/s]

New best score: -547.39
New best score: -520.48



  1%|          | 7/1000 [00:01<04:23,  3.77it/s]
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/site-packages/tqdm/_tqdm.py", line 103, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  1%|          | 8/1000 [00:02<04:13,  3.91it/s]

New best score: -458.38


  5%|▌         | 52/1000 [00:11<03:17,  4.80it/s]

New best score: -431.02


 99%|█████████▊| 986/1000 [03:47<00:03,  3.77it/s]

New best score: -404.38


100%|██████████| 1000/1000 [03:51<00:00,  2.69it/s]
